# Programación Dinámica

Inspirado por [Principles of Algorithmic Problem Solving](http://csc.kth.se/~jsannemo/slask/main.pdf) de Johan Sannemo.

La **programación dinámica** es recursión + memoization.

## Change problem

Esto se puede ver en un caso sencillo con Fibonacci, pero se ilustra mucho mejor con el problema del "cambio" (expresar una cantidad con monedas de diferentes valores). Cuando los valores son por ejemplo (1,2,5) el método greedy es óptimo, pero hay otros casos como (1,6,7) donde es necesario explorar. Por ejemplo: 12=6+6, pero si empezamos por el mayor tenemos una solución peor: 12=7+1+1+1+1+1.

La forma de resolverlo es la siguiente. Si tenemos que formar la cantidad $T$, tenemos 3 posibilidades:

- usar 7 y formar $T-7$
- usar 6 y formar $T-6$
- usar 1 y formar $T-1$

Calculamos las 3 y elegimos el subproblema que necesite menos monedas en total. El resultado será añadir al subproblema elegido la moneda usada.

Funciona perfecto. Pero, como ocurre en Fibonacci, se repiten muchos subproblemas. La solución es inmediata: *memoization*.

Se guardan los casos calculados para que cuando vuelvan a necesitarse estén disponibles directamente. Interesa que los argumentos de la función sean mínimos y apropiados para que no se dispare el número de casos.

Se puede hacer top-down (recursión natural) o bottom-up (generando todos los casos hasta llegar al de interés, puede que alguno no nos haga falta).

TODO: versión base, añadir contador de llamadas recursivas, después versión memoizada, y después problema de recursion limit.

In [ ]:
from functools import lru_cache

D = [1,6,7]

@lru_cache(maxsize=None)
def change(t):
    global nc
    nc += 1
    if t==0:
        return 0, []
    pos = [ (change(t-d),d) for d in D if t>=d ]
    (n,l),s = min(pos)
    return 1+n, l+[s]

nc = 0  # counter of calls

In [ ]:
change(17)

In [ ]:
for k in range(50):
    nc = 0
    print(k, change(k), nc)

Si pedimos un valor grande superamos el límite de recursión. Podemos aumentarlo con `sys.setrecursionlimit()`.

## Smith-Waterman

The [Smith-Waterman algorithm](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm) for sequence alignment:

In [ ]:
import numpy as np

def s(x,y):
    return 3 if x==y else -3

def w(k):
    return 2*k


def SM(b,a,S,W):
    na = len(a)
    nb = len(b)
    H = np.zeros((na+1, nb+1))
    for i in range(1,na+1):
        for j in range(1,nb+1):
            H[i,j] = max([0,
                          H[i-1,j-1] + s(a[i-1],b[j-1]),
                          max( [H[i-k,j]-w(k) for k in range(1,i+1)] ),
                          max( [H[i,j-l]-w(l) for l in range(1,j+1)] )
                         ])
    print(H)
    print(np.max(H))
    I,J = np.where(H==np.max(H))
    print(H[I[0],J[0]])
    sols = []
    for i,j in zip(I,J):
        print(i,j)
        sol = []
        #print(a)
        #print(b)
        sol.append( (H[i,j],(i,j),(a[i-1],b[j-1])) )
        while True:
            
            x = _,(i,j),_ = max([ (H[i-1,  j], (i-1,j) ,   (a[i-2],b[j-1]+'-')) ,
                                  (H[i-1,j-1], (i-1,j-1) , (a[i-2],b[j-2])) ,
                                  (H[i,  j-1], (i,j-1),    (a[i-1]+'-',b[j-2])) ] )
            if H[i,j]==0: break
            sol.append(x)
        
        bl,al = zip(*[k[-1] for k in reversed(sol)])
        sols.append( (fixmissing(al), fixmissing(bl)) )
    return sols


def fixmissing(x):
    a = list(x)
    for k in reversed(range(1,len(a))):
        if a[k-1][-1] == '-':
            a[k] = '-'
        else:
            if a[k][-1] == '-':
                a[k] = a[k][:-1]
    if a[0][-1] == '-':
        a[0] = a[0][:-1]
    return ''.join(a)

In [ ]:
SM('xjAlberto','jHAbeton',s,w)

In [ ]:
SM('TGTTACGG', 'GGTTGACTA', s, w)

In [ ]:
kk=SM('991230568955512345678911', '234567895551235608', s, w)
for a,b in kk:
    print(a)
    print(b)
    print()